# 02. Preprocessing (전처리)

## 목표
- QC (품질 관리): 응답 부족, Straight-lining 제외
- Big Five + Ideology + Honesty-Humility 점수 계산

## 참조 파일
- `reports/preprocessing_guide.md` - 점수 계산 공식
- `reports/step1_scan.json` - Step 1 결과

In [1]:
%pip install pandas numpy -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

# 작업 디렉토리 설정
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')

작업 폴더: c:\Users\yujin\code\agent_seminar\sapa-demo


In [3]:
# Step 1 결과 로드
with open('reports/step1_scan.json', 'r', encoding='utf-8') as f:
    step1 = json.load(f)
print(f"Step 1 결과: {step1['results']['n_respondents']:,}명, {step1['results']['n_items']}문항")

Step 1 결과: 23,679명, 696문항


In [4]:
# 데이터 로드
data = pd.read_csv('data/raw/sapa_data.csv')
keys = pd.read_csv('data/raw/superKey696.csv', index_col=0)

# 문항 컬럼 식별
item_cols = [col for col in data.columns if col.startswith('q_')]
print(f"데이터 로드: {len(data):,}명, {len(item_cols)}문항")

데이터 로드: 23,679명, 696문항


---
## Part 1: QC (Quality Control)

In [5]:
# QC 기준 (고정값)
MIN_RESPONSES = 10

# 1. 응답 부족: 10개 미만 응답
responses_per_person = data[item_cols].notna().sum(axis=1)
low_response_mask = responses_per_person < MIN_RESPONSES
low_response_ids = data.loc[low_response_mask, 'RID'].tolist()

# 2. Straight-lining: 모든 응답이 동일한 값
def is_straight_lining(row):
    valid_responses = row.dropna()
    if len(valid_responses) < 10:
        return False
    return valid_responses.nunique() == 1

straight_line_mask = data[item_cols].apply(is_straight_lining, axis=1)
straight_line_ids = data.loc[straight_line_mask, 'RID'].tolist()

# 제외 대상 합산
exclude_ids = set(low_response_ids) | set(straight_line_ids)

# QC 적용
data_clean = data[~data['RID'].isin(exclude_ids)].copy()

print("=" * 50)
print("=== QC 결과 ===")
print("=" * 50)
print(f"응답 부족 ({MIN_RESPONSES}개 미만): {len(low_response_ids)}명")
print(f"Straight-lining: {len(straight_line_ids)}명")
print(f"제외 합계: {len(exclude_ids)}명")
print(f"유효 응답자: {len(data_clean):,}명")
print("=" * 50)

=== QC 결과 ===
응답 부족 (10개 미만): 1명
Straight-lining: 31명
제외 합계: 32명
유효 응답자: 23,647명


응답 부족 (10개 미만): 1명
비율: 0.00%


---
## Part 2: Big Five 점수 계산

In [6]:
def calculate_scale_score(df, keys, scale_name):
    """
    채점 키를 사용해 척도 점수 계산
    - 1: 정채점, -1: 역채점 (7 - 원점수), 0: 해당 없음
    """
    # 해당 척도에 속하는 문항 찾기
    scale_items = keys.index[keys[scale_name] != 0].tolist()
    weights = keys.loc[scale_items, scale_name]
    
    # 데이터에 있는 문항만 필터링
    available_items = [q for q in scale_items if q in df.columns]
    
    if not available_items:
        return pd.Series([np.nan] * len(df), index=df.index)
    
    # 역채점 적용 (6점 척도: 7 - 원점수)
    subset = df[available_items].copy()
    for item in available_items:
        if weights[item] == -1:
            subset[item] = 7 - subset[item]
    
    # 평균 계산 (결측 무시)
    return subset.mean(axis=1, skipna=True)

print("척도 점수 계산 함수 정의 완료")

척도 점수 계산 함수 정의 완료


In [7]:
# Big Five 점수 계산
scores = pd.DataFrame()
scores['RID'] = data_clean['RID'].values

big_five_scales = ['NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N']

print("Big Five 점수 계산 중...\n")
print("=== Big Five 점수 결과 ===")

for scale in big_five_scales:
    scores[scale] = calculate_scale_score(data_clean, keys, scale).values
    valid_n = scores[scale].notna().sum()
    mean_val = scores[scale].mean()
    std_val = scores[scale].std()
    print(f"{scale}: N={valid_n:,}, M={mean_val:.2f}, SD={std_val:.2f}")

Big Five 점수 계산 중...

=== Big Five 점수 결과 ===
NEO_O: N=23,399, M=4.31, SD=0.83
NEO_C: N=23,417, M=4.23, SD=0.86
NEO_E: N=23,357, M=3.87, SD=0.92
NEO_A: N=23,437, M=4.23, SD=0.82
NEO_N: N=23,371, M=3.35, SD=0.98


---
## Part 3: Ideology 점수 계산

**공식**: `Ideology = mean(z(MPQtr), z(NEOo6) * -1)`

In [8]:
def z_score(series):
    """표준화 (z-score)"""
    return (series - series.mean()) / series.std()

# 하위 척도 계산
mpq_tr = calculate_scale_score(data_clean, keys, 'MPQtr')
neo_o6 = calculate_scale_score(data_clean, keys, 'NEOo6')

# Ideology = mean(z(MPQtr), z(NEOo6) * -1)
# 중요: .values로 index 정보 제거 (scores DataFrame과 index 불일치 방지)
scores['Ideology'] = ((z_score(mpq_tr) + z_score(neo_o6) * -1) / 2).values

valid_n = scores['Ideology'].notna().sum()
mean_val = scores['Ideology'].mean()
std_val = scores['Ideology'].std()

print("\n=== Ideology 점수 결과 ===")
print(f"Ideology: N={valid_n:,}, M={mean_val:.3f}, SD={std_val:.3f}")


=== Ideology 점수 결과 ===
Ideology: N=11,810, M=0.033, SD=0.969


---
## Part 4: Honesty-Humility 점수 계산

**공식**: `Honesty_Humility = mean(z(NEOa2), z(NEOa4), z(HEXACO_H))`

In [9]:
# 하위 척도 계산
neo_a2 = calculate_scale_score(data_clean, keys, 'NEOa2')
neo_a4 = calculate_scale_score(data_clean, keys, 'NEOa4')
hexaco_h = calculate_scale_score(data_clean, keys, 'HEXACO_H')

# Honesty_Humility = mean(z(NEOa2), z(NEOa4), z(HEXACO_H))
# 중요: .values로 index 정보 제거 (scores DataFrame과 index 불일치 방지)
scores['Honesty_Humility'] = ((z_score(neo_a2) + z_score(neo_a4) + z_score(hexaco_h)) / 3).values

valid_n = scores['Honesty_Humility'].notna().sum()
mean_val = scores['Honesty_Humility'].mean()
std_val = scores['Honesty_Humility'].std()

print("\n=== Honesty-Humility 점수 결과 ===")
print(f"Honesty_Humility: N={valid_n:,}, M={mean_val:.3f}, SD={std_val:.3f}")


=== Honesty-Humility 점수 결과 ===
Honesty_Humility: N=12,811, M=-0.003, SD=0.727


---
## Part 5: 결과 저장

In [10]:
# processed 폴더 생성
os.makedirs('data/processed', exist_ok=True)

# 최종 점수 데이터 (RID + 7개 척도)
final_cols = ['RID', 'NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N', 'Ideology', 'Honesty_Humility']
scores_final = scores[final_cols]

# 저장
scores_final.to_csv('data/processed/sapa_scores.csv', index=False)

print("\n" + "=" * 50)
print("=== 점수 계산 결과 요약 ===")
print("=" * 50)
for col in final_cols[1:]:
    n = scores_final[col].notna().sum()
    m = scores_final[col].mean()
    sd = scores_final[col].std()
    print(f"{col}: N={n:,}, M={m:.2f}, SD={sd:.2f}")
print("=" * 50)
print(f"\n✅ 저장 완료: data/processed/sapa_scores.csv")
print(f"   총 {len(scores_final):,}명, {len(final_cols)-1}개 척도")


=== 점수 계산 결과 요약 ===
NEO_O: N=23,399, M=4.31, SD=0.83
NEO_C: N=23,417, M=4.23, SD=0.86
NEO_E: N=23,357, M=3.87, SD=0.92
NEO_A: N=23,437, M=4.23, SD=0.82
NEO_N: N=23,371, M=3.35, SD=0.98
Ideology: N=11,810, M=0.03, SD=0.97
Honesty_Humility: N=12,811, M=-0.00, SD=0.73

✅ 저장 완료: data/processed/sapa_scores.csv
   총 23,647명, 7개 척도
